In [10]:
import gym
from gym import spaces
import gym_ple
import numpy as np

### Flappy Bird Sem Visão

Definição do Ambiente

In [11]:
class FlappyBirdEnv(gym.Env):

  def __init__(self):
    self.env = gym.make("FlappyBird-v0")
    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions:
    self.action_space = self.env.action_space
    # Example for using image as input:
    self.observation_space = spaces.Box(low=np.array([0, -10.0, 0, 0, 0, 0, 0, 0]),
                                        high=np.array([512, 10.0, 588.0, 512, 512, 588.0, 512, 512]),
                                        dtype=np.float32)

  def step(self, action):
    observation, reward, done, info = self.env.step(action)
    observation = np.array(list(self.env.game_state.getGameState().values()))
    if done:
        reward = -1
    reward += 0.1
#     reward += (75 - abs(observation[0] - (observation[3] + observation[4])/2))*max((300 - observation[2])/300, 0)/750
    return observation, reward, done, info

  def reset(self):
    self.env.reset()
    observation = np.array(list(self.env.game_state.getGameState().values()))
    return observation  # reward, done, info can't be included

  def render(self):
    self.env.render()
    
  def close (self):
    self.env.close()

In [12]:
from stable_baselines.common.env_checker import check_env

env = FlappyBirdEnv()
check_env(env)

In [13]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import set_global_seeds, make_vec_env
# from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

env = make_vec_env(FlappyBirdEnv, n_envs = 16)
# Optional: PPO2 requires a vectorized environment to run
# the env is now wrapped automatically when passing it to the constructor
# env = DummyVecEnv([lambda: env])

In [14]:
model = PPO2(MlpPolicy, env, n_steps = 512, nminibatches = 64, lam = 0.98, gamma = 0.99, noptepochs= 10, ent_coef= 0.001, verbose=1)

In [15]:
model = PPO2.load("trained_models/PPO2")
model.set_env(env)

Loading a model without an environment, this model cannot be trained until it has a valid environment.


In [37]:
model.learn(total_timesteps=100000)

-------------------------------------
| approxkl           | 0.018743424  |
| clipfrac           | 0.08632813   |
| ep_len_mean        | 618          |
| ep_reward_mean     | 107          |
| explained_variance | 0.59         |
| fps                | 338          |
| n_updates          | 1            |
| policy_entropy     | 0.20740166   |
| policy_loss        | 0.0007544595 |
| serial_timesteps   | 512          |
| time_elapsed       | 0            |
| total_timesteps    | 8192         |
| value_loss         | 1.624325     |
-------------------------------------
------------------------------------
| approxkl           | 0.015732259 |
| clipfrac           | 0.08270264  |
| ep_len_mean        | 616         |
| ep_reward_mean     | 107         |
| explained_variance | 0.462       |
| fps                | 340         |
| n_updates          | 2           |
| policy_entropy     | 0.21342364  |
| policy_loss        | 0.003289494 |
| serial_timesteps   | 1024        |
| time_elapsed       | 

---------------------------------------
| approxkl           | 0.02410226     |
| clipfrac           | 0.07977295     |
| ep_len_mean        | 905            |
| ep_reward_mean     | 162            |
| explained_variance | 0.399          |
| fps                | 322            |
| n_updates          | 16             |
| policy_entropy     | 0.24261951     |
| policy_loss        | -0.00018298425 |
| serial_timesteps   | 8192           |
| time_elapsed       | 376            |
| total_timesteps    | 131072         |
| value_loss         | 2.439866       |
---------------------------------------
-------------------------------------
| approxkl           | 0.02590335   |
| clipfrac           | 0.08952637   |
| ep_len_mean        | 900          |
| ep_reward_mean     | 161          |
| explained_variance | 0.279        |
| fps                | 326          |
| n_updates          | 17           |
| policy_entropy     | 0.26802757   |
| policy_loss        | 3.569468e-05 |
| serial_timesteps  

KeyboardInterrupt: 

In [6]:
flappyTest = FlappyBirdEnv()

In [9]:
obs = flappyTest.reset()
dones = 0
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = flappyTest.step(action)
    flappyTest.render()

flappyTest.close()

In [39]:
model.save("trained_models/PPO2")

### Flappy Bird CNN LSTM

In [11]:
ENV_NAME = "FlappyBird-v0"

import gym

from stable_baselines.common.policies import CnnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

env_vision = gym.make(ENV_NAME)
# Optional: PPO2 requires a vectorized environment to run
# the env is now wrapped automatically when passing it to the constructor
env_vision = DummyVecEnv([lambda: env_vision])

In [ ]:
modelVision = PPO2(CnnLstmPolicy, env_vision, n_steps = 512, nminibatches = 1, lam = 0.98, gamma = 0.999, noptepochs= 15, ent_coef= 0.01, verbose=1)

In [ ]:
modelVision.learn(total_timesteps=10000)

In [ ]:
obs = env_vision.reset()
dones = 0
while not dones:
    action, _states = modelVision.predict(obs)
    obs, rewards, dones, info = env_vision.step(action)
    env.render()

env_vision.close()